In [1]:
import pandas as pd
import numpy as np

In [2]:
# tuning
tuning=pd.read_csv("result/gold_in_qa_split/tuning.txt",sep=" ")

In [3]:
cumu_time=tuning["time"]

In [4]:
excute_time=[ cumu_time[i] if i==0 else cumu_time[i]-cumu_time[i-1] for i in range(len(cumu_time)) ]

In [5]:
tuning["excute_time"]=excute_time

In [6]:
tuning.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
0,GBDTBTt,0.0,0.10,0.0,50,20,0.315876,150,150
1,GBDTBTt,0.0,0.10,0.0,100,20,0.318941,305,155
2,GBDTBTt,0.0,0.10,0.0,200,20,0.318260,646,341
3,GBDTBTt,0.0,0.05,0.0,50,20,0.313231,716,70
4,GBDTBTt,0.0,0.05,0.0,100,20,0.312098,854,138


In [7]:
tuning.sort_values(by="RMSE",inplace=True)

In [8]:
BTt=tuning[tuning["name"]=="GBDTBTt"]

In [9]:
GBFS=tuning[tuning["name"]=="GBFS_model"]
GBFS_adapt=tuning[tuning["name"]=="GBFS_adapt_model"]

In [10]:
GBFS["excute_time"].mean()

159.15555555555557

In [11]:
GBFS_adapt["excute_time"].mean()

155.8266129032258

In [12]:
BTt["excute_time"].mean()

80.75

In [13]:
# tuning plots
GBFS.head()


,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
686,GBFS_model,8.0,0.10,0.05,200,30,0.294307,112401,231
377,GBFS_model,2.0,0.05,0.01,200,51,0.294337,71753,264
530,GBFS_model,4.0,0.10,0.02,200,39,0.294592,92412,258
662,GBFS_model,2.0,0.10,0.05,200,45,0.295124,109310,230
818,GBFS_model,4.0,0.10,0.10,200,34,0.295218,128054,210


In [14]:
GBFS_adapt.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
590,GBFS_adapt_model,0.01,0.10,0.05,200,42,0.294803,99746,230
305,GBFS_adapt_model,0.01,0.05,0.01,200,60,0.295456,61380,270
341,GBFS_adapt_model,0.16,0.05,0.01,200,32,0.295826,66506,253
449,GBFS_adapt_model,0.01,0.05,0.02,200,53,0.295904,81150,248
589,GBFS_adapt_model,0.01,0.10,0.05,100,42,0.296298,99516,115


In [15]:
BTt.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
722,GBDTBTt,0.0,0.10,0.10,200,20,0.298999,116880,93
437,GBDTBTt,0.0,0.05,0.02,200,20,0.299033,79928,105
581,GBDTBTt,0.0,0.05,0.05,200,20,0.299320,99000,97
578,GBDTBTt,0.0,0.10,0.05,200,20,0.299435,98821,100
434,GBDTBTt,0.0,0.10,0.02,200,20,0.299815,79739,110


In [12]:
## analysis tuned result for gold_in_qa_split
from os import listdir
from sklearn.metrics import roc_auc_score


In [13]:
def ranking_precision_score(y_true, y_score, k=10):
    """Precision at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    Returns
    -------
    precision @k : float
    """
    unique_y = np.unique(y_true)

    if len(unique_y) == 2:
        pos_label = unique_y[1]
        n_pos = np.sum(y_true == pos_label)

        order = np.argsort(y_score)[::-1]
        y_true = np.take(y_true, order[:k])
        n_relevant = np.sum(y_true == pos_label)

        # Divide by min(n_pos, k) such that the best achievable score is always 1.0.
        return float(n_relevant) / min(n_pos, k)
    else:
        return -1
def rank_precision(data_frame,k=4): # passed to groupby object 
    y_true=list(data_frame[1])
    y_score=list(data_frame[2])
    return ranking_precision_score(y_true, y_score, k=k)

In [14]:
class Model:
    def __init__(self,name=None,alpha=0,penalty=0,topk=-1,attrN=-1,shrink=0.1,
                 itern=100,preds=None,rmse=-1):
        self.name=name  # name can be GBDT or GBDTBT (GBDT with top ranking feature from BT)
        self.pen=penalty
        self.topk=topk
        self.preds=preds
        self.attrN=attrN
        self.alpha=alpha
        self.shrink=shrink
        self.itern=itern
        self.rmse=rmse
        
    def get_AUC(self):
        return roc_auc_score(trueY,self.preds) if self.preds is not None else None
    
    #add some other metrics later
    
    def get_ranking_precision(self,k=2): # ranking_precision_score averaged over queries
        combined=pd.concat([kw,trueY,self.preds],axis=1,ignore_index=True)
        grouped=combined.groupby(combined[0])
        agged=grouped.apply(rank_precision,k=k)
        agged=[num for num in agged if num>=0]
        return sum(agged)/(len(agged)) if len(agged)>0 else 0
        

In [15]:
## result path

# result_path='result/gold_in_qa_split/tuned/'
# attr_total=96
result_path='result/us/tuned/'
attr_total=174

In [16]:
filenames_result=listdir(result_path)
trueY=pd.read_csv(result_path+"trueY.txt",header=None)[0]
try:
    kw=pd.read_csv(result_path+"keyword.txt",header=None)[0]
except:
    print("do not have groups")
try:
    number_unused=len(pd.read_csv(result_path+"preprocess_unused.txt",header=None))
except:
    number_unused=0
attr_init_number=attr_total-number_unused

In [17]:
filenames_result

['GBDTBTt_preds_mu0_alpha0.02_shrink0.05_itern200_attrn20_rms0.23687.txt',
 'us_test.dta',
 'feature_scores_BT_alpha0.02.txt',
 'GBFS_model_preds_mu16_alpha0.02_shrink0.1_itern100_attrn13_rms0.235527.txt',
 'trueY.txt',
 'preprocess_unused.txt',
 'GBDTBTt_preds_mu0_alpha0.1_shrink0.1_itern200_attrn20_rms0.244205.txt',
 'GBFS_model_preds_mu8_alpha0.1_shrink0.1_itern200_attrn23_rms0.233999.txt',
 'GBFS_model_preds_mu8_alpha0.05_shrink0.1_itern200_attrn20_rms0.233055.txt',
 'feature_scores_BT_alpha0.1.txt',
 'GBFS_model_preds_mu8_alpha0.02_shrink0.1_itern200_attrn20_rms0.233091.txt',
 'keyword.txt',
 'us_train.attr',
 'tuned.txt',
 'us_train.dta',
 'GBFS_model_preds_mu8_alpha0.05_shrink0.1_itern100_attrn20_rms0.232841.txt']

In [18]:
import collections
results_dict=collections.defaultdict(list)# name,mu,alpha,shrink,itern,attrn,rmse,auc,prec@k=2,prec@k=3,prec@k=4

In [19]:
models=[]

In [20]:
for fname in filenames_result:
    last_dot=fname.rfind(".")
    filename=fname[:last_dot]
    if "_preds" in filename:
        name=filename[:filename.index("_preds")]
        mu=float(filename[filename.index("_mu")+3:filename.index("_alpha")])
        alpha=float(filename[filename.index("_alpha")+6:filename.index("_shrink")])
        shrink=float(filename[filename.index("_shrink")+7:filename.index("_itern")])
        itern=int(filename[filename.index("_itern")+6:filename.index("_attrn")])
        attrn=int(filename[filename.index("_attrn")+6:filename.index("_rms")])
        rms=float(filename[filename.index("_rms")+4:])
        preds=pd.read_csv(result_path+fname,header=None)[0]
        models.append(Model(name=name,alpha=alpha,penalty=mu,attrN=attrn,shrink=shrink,
                 itern=itern,preds=preds,rmse=rms))
        
    

In [21]:
for model in models:
    results_dict["name"].append(model.name)
    results_dict["mu"].append(model.pen)
    results_dict["shrink"].append(model.shrink)
    results_dict["itern"].append(model.itern)
    results_dict["attrn"].append(model.attrN)
    results_dict["alpha"].append(model.alpha)
    results_dict["rmse"].append(model.rmse)
    results_dict["AUC_ROC"].append(model.get_AUC())
    results_dict["prec@k=2"].append(model.get_ranking_precision(k=2))
    results_dict["prec@k=3"].append(model.get_ranking_precision(k=3))
    results_dict["prec@k=4"].append(model.get_ranking_precision(k=4))
    
    
    
    

In [22]:
results_table=pd.DataFrame(results_dict)

In [24]:
results_table.sort_values("name",inplace=True)

In [25]:
results_table

,name,mu,shrink,itern,attrn,alpha,rmse,AUC_ROC,prec@k=2,prec@k=3,prec@k=4
0,GBDTBTt,0.0,0.05,200,20,0.02,0.236870,0.891337,0.896104,0.954545,0.984848
2,GBDTBTt,0.0,0.10,200,20,0.10,0.244205,0.857397,0.887446,0.944444,0.971501
1,GBFS_model,16.0,0.10,100,13,0.02,0.235527,0.900230,0.913420,0.962482,0.984127
3,GBFS_model,8.0,0.10,200,23,0.10,0.233999,0.919336,0.913420,0.954545,0.979076
4,GBFS_model,8.0,0.10,200,20,0.05,0.233055,0.913622,0.906926,0.950938,0.979798
5,GBFS_model,8.0,0.10,200,20,0.02,0.233091,0.873678,0.898268,0.945887,0.980519
6,GBFS_model,8.0,0.10,100,20,0.05,0.232841,0.919979,0.911255,0.951659,0.980880


In [25]:
# make entire query asin pair all in training or testing to do ranked precision/recall
data_path="data/"
data_name="gold_in_qa_split"

In [26]:
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
test=pd.read_csv(data_path+data_name+"_test.tsv",sep="\t")

In [27]:
# random split 10% train to form validation dataset according to query

In [28]:
queries_train=train["keyword"].unique()

In [30]:
len(queries_train_validate)

767

In [29]:
validate_data=train[[query in queries_train_validate for query in train["keyword"]]]

In [8]:
queries_train=train["keyword"].unique()

In [33]:
train_data=train[[query not in queries_train_validate for query in train["keyword"]]]

In [34]:
len(validate_data)

10920

In [35]:
len(train_data)

96312

In [36]:
validate_data.to_csv(data_path+data_name+"_train_validate.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train_train.tsv",sep="\t",index=False)

In [14]:
all_data=pd.concat([train,test],ignore_index=True)

In [15]:
queries=all_data["keyword"].unique()

In [16]:
len(all_data)/len(queries)

3.8241020476670022

In [106]:
np.random.seed(1)
test_queries=set(np.random.choice(queries,replace=False,size=(len(test)*len(queries))//len(all_data)))

In [107]:
test_data=all_data[[query in test_queries for query in all_data["keyword"]]]

In [108]:
train_data=all_data[[query not in test_queries for query in all_data["keyword"]]]

In [109]:
len(test_data)/len(train_data),len(test)/len(train)

(0.11747138964577657, 0.11019913157410315)

In [110]:
test_data.to_csv(data_path+data_name+"_test.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train.tsv",sep="\t",index=False)

In [ ]:
## madelon dataset

In [4]:
data_path="data/"
data_name="madelon"

In [5]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [6]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,-1,1:485,2:477,3:537,4:479,5:452,6:471,7:491,8:476,9:475,...,491:477,492:481,493:477,494:485,495:511,496:485,497:481,498:479,499:475,500:496
1,-1,1:483,2:458,3:460,4:487,5:587,6:475,7:526,8:479,9:485,...,491:463,492:478,493:487,494:338,495:513,496:486,497:483,498:492,499:510,500:517
2,-1,1:487,2:542,3:499,4:468,5:448,6:471,7:442,8:478,9:480,...,491:487,492:481,493:492,494:650,495:506,496:501,497:480,498:489,499:499,500:498
3,1,1:480,2:491,3:510,4:485,5:495,6:472,7:417,8:474,9:502,...,491:491,492:480,493:474,494:572,495:454,496:469,497:475,498:482,499:494,500:461
4,1,1:484,2:502,3:528,4:489,5:466,6:481,7:402,8:478,9:487,...,491:488,492:479,493:452,494:435,495:486,496:508,497:481,498:504,499:495,500:511


In [7]:
train_data=train.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [8]:
train_data[0]=[ int(x>0) for x in train[0]]

In [9]:
test_data=test.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [10]:
test_data[0]=[ int(x>0) for x in test[0]]

In [11]:
test_data.rename(columns={0:"label"},inplace=True)

In [12]:
train_data.rename(columns={0:"label"},inplace=True)

In [15]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [ ]:
## gisette_scale data

In [17]:
data_path="data/"
data_name="gisette_scale"

In [18]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [80]:
train_data=train.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [84]:
train_data[0]=[ int(x>0) for x in train_data[0]]

In [85]:
test_data=test.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [86]:
test_data[0]=[ int(x>0) for x in test_data[0]]

In [92]:
# test_data has more columns than train_data?

In [93]:
test_data.rename(columns={0:"label"},inplace=True)
train_data.rename(columns={0:"label"},inplace=True)

In [97]:
test_data=test_data.iloc[:,:4957]

In [99]:
test_data.shape,train_data.shape

((1000, 4957), (6000, 4957))

In [100]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [2]:
train="data/gisette_scale_train.tsv"
test="data/gisette_scale_test.tsv"
response_name="label"
folder="result/gisette_scale"

In [3]:
train=pd.read_csv(train,sep="\t")
train_names=list(train.columns.values)
file_names=listdir(folder)

In [6]:
allfeature=set(pd.read_csv(folder+"/allfeature.txt",header=None)[0])

In [12]:
set(str(x) for x in allfeature)

{'1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [15]:
len(us_allfeature)

98

In [22]:
pd.DataFrame(us_allfeature).to_csv("result/us/allfeature.txt",header=None,index=None)

In [42]:
cpod_trueY=pd.read_csv("result/cpod/trueY.txt",header=None)[0]

In [43]:
cpod_pred=pd.read_csv("result/cpod/preds.txt",header=None)[0]

In [44]:
cpod_pred20=pd.read_csv("result/cpod/preds_top20.txt",header=None)[0]

In [45]:
cpod_pred.head()

0   -0.000588
1   -0.000588
2   -0.000758
3   -0.000758
4   -0.000588
Name: 0, dtype: float64

In [46]:
len(cpod_pred20)

40789868

In [51]:
rmse=(((cpod_pred-cpod_trueY)**2).sum()/len(cpod_pred20))**0.5

In [52]:
rmse20=(((cpod_pred20-cpod_trueY)**2).sum()/len(cpod_pred20))**0.5

In [53]:
rmse20

0.11185116617780548

In [54]:
rmse

0.11214546111170468